## Elements

In [0]:
import pandas as pd

In [0]:
class Element:


  def __init__(self, name, abrivation, atomic_number,
               atomic_mass, period, group):
    
    self.name = name
    self.abrivation = abrivation
    self.atomic_number = atomic_number
    self.atomic_mass = atomic_mass
    self.period = period #row
    self.group = group #column
    self.protons = self.atomic_number
    self.neutrons = self.atomic_mass - self.protons


  def __repr__(self):
    return f"{self.abrivation}\n{self.atomic_number}\n{self.atomic_mass}"

In [0]:
df = pd.read_csv('elements.csv', header=None).dropna(axis = 0)
df.head()

,0,1,2,3,4,5
0,'Hydrogen','H',1.0,1.008,1.0,1.0
1,'Helium','He',2.0,4.003,1.0,18.0
2,'Lithium','Li',3.0,6.941,2.0,1.0
3,'Beryllium','Be',4.0,9.012,2.0,2.0
4,'Boron','B',5.0,10.811,2.0,13.0


In [0]:
df[0] = df[0].str.strip("'")
df[1] = df[1].str.strip("'")

In [0]:
elements = {}

for i in range(len(df)):
  element = Element(name = df[0][i],
                    abrivation = df[1][i],
                    atomic_number = df[2][i],
                    atomic_mass = df[3][i],
                    period = df[4][i],
                    group = df[5][i])
  

  elements[element.abrivation] = {'name': element.name,
                    'atomic_number': element.atomic_number,
                    'atomic_mass': element.atomic_mass,
                    'period':element.period,
                    'group': element.group}

In [0]:
elements['H']

{'atomic_mass': 1.008,
 'atomic_number': 1.0,
 'group': 1.0,
 'name': 'Hydrogen',
 'period': 1.0}

In [0]:
hydrogen = Element(name = 'hydrogen',abrivation = 'H',
                   atomic_number = 1, atomic_mass = 1.0079,
                   period = 1, group = 1)


hydrogen

H
1
1.0079

## Balencing Equations

In [0]:
import pandas as pd
import numpy as np


def balence(equation_str):
  return_dict = {}
  for species in equation.replace('->', '+').split('+'):
    return_dict[species.strip()] = {}
    for element in species.strip().split(' '):
      if len(element) < 3:
        element += '_1'
      return_dict[species.strip()][element.split('_')[0]] = element.split('_')[1]


  # Lawrence R. Thorne: Simplified Matrix Null-Space Method

  # Step 0
  chem_comp_table = pd.DataFrame(return_dict, dtype = 'float').fillna(0)
  #print("Step 0\n", chem_comp_table)
  #print('\n')

  # Step 1
  chem_comp_matrix = chem_comp_table.values
  #print("Step 1\n", chem_comp_matrix)
  #print('\n')

  # Step 2
  nullity = len(chem_comp_table.columns) - np.linalg.matrix_rank(chem_comp_matrix)
  if nullity == 0:
    return "No solution"
  #print("Step 2\n", nullity)
  #print('\n')

  # Step 3
  a = np.zeros((nullity, chem_comp_matrix.shape[1]))
  b = np.identity(nullity)
  augmentation = np.append(a[:, :a.shape[1] - b.shape[1]], b, axis = 1)
  augmented_ccm = np.append(chem_comp_matrix, augmentation, axis = 0)
  #print("Step 3\n", augmented_ccm)
  #print('\n')

  # Step 4
  inverted_ccm = np.linalg.inv(augmented_ccm)
  #print("Step 4\n", inverted_ccm)
  #print('\n')

  # Step 5 & 6
  null_space_vec = inverted_ccm[:, -nullity].T
  #print("Step 5 & 6\n", null_space_vec)
  #print('\n')

  # Step 7
  coefs = null_space_vec / min([abs(i) for i in list(null_space_vec)])
  if (float("inf") in coefs) or (float("-inf") in coefs):
    return "Infinite solutions"
  coefs = [int(round(abs(i))) for i in coefs]
  #print("Step 7\n", coefs)
  #print('\n')

  # Step 8
  species = [i.strip() for i in equation.replace('->', '+').split('+')]
  for i in range(len(species)):
    return_dict[species[i]] = coefs[i]

  return_eq = []

  for side in equation.split('->'):
    new_side = []
    for species in side.split('+'):
      #print(return_dict[species.strip()], species.strip())
      new_side.append(str(return_dict[species.strip()]) + str(species.strip()))
    return_eq.append(' + '.join(new_side))

  return ' -> '.join(return_eq)

In [0]:
equation = 'C H_4 + H_2 O -> C O_2 + H_2'

balence(equation)

'1C H_4 + 2H_2 O -> 1C O_2 + 4H_2'

In [0]:
equation = 'K I + K Cl O_3 + H Cl -> I_2 + H_2 O + K Cl'

balence(equation)

'6K I + 1K Cl O_3 + 6H Cl -> 3I_2 + 3H_2 O + 7K Cl'

In [0]:
equation = 'C O + C O_2 + H_2 -> C H_4 + H_2 O'

balence(equation)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:53: RuntimeWarning: divide by zero encountered in true_divide
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:53: RuntimeWarning: invalid value encountered in true_divide


'Infinite solutions'

In [0]:
equation = 'Fe S_2 + H N O_3 -> Fe_2 S_3 O_12 + N O + H_2 S O_4'

balence(equation)

'No solution'

In [0]:
#Sabitier raction
equation = 'C O_2 + H_2 -> C H_4 + O_2'

balence(equation)

'1C O_2 + 2H_2 -> 1C H_4 + 1O_2'

## Stoichiometry

In [0]:
avogadro = 6.0221409E23


def atoms_to_moles(atoms):
  return atoms / avogadro


def moles_to_mass(moles, molar_mass):
  return moles * molar_mass


def mass_to_moles(mass, molar_mass):
  return mass / molar_mass


def moles_to_atoms(moles):
  return moles * avogadro

In [0]:
moles_to_atoms(5E-10)

## Calculating percent composition

In [0]:
def chem_spliter(chem):
  """
  Input: Chemical formula in form (H_2 O)
  Output: Array of tuples shape (Chem abbv, ,moles)
  """
  return_array = []
  for i in chem.split(' '):
    return_array.append((i.split('_')[0],i.split('_')[1]))
  return return_array

In [0]:
def percent_composition(chemical):
  """
  Input: Chemical formula in form (H_2 O)
  Output: Dict in form {Element: %comp}
  """
  return_dict = {}
  masses = []
  chem_spliter(chemical)

  for i in chem_spliter(chemical):
    masses.append(elements[i[0]]['atomic_mass'] * int(i[1]))

  chemical_mass = sum(masses)

  for k, v in zip(chem_spliter(chemical), masses):
    return_dict[k[0]] = (v / chemical_mass) * 100

  return return_dict

In [0]:
chemical = 'C_10 F_18'

percent_composition(chemical)